In [1]:
import itertools as it

import pandas as pd
from sqlalchemy.orm import Session

import src.db.models.doccano as m
from src.coding.labels import LABELS
from src.coding.vis import visualize_dataframe
from src.db import load
from src.db.connect import make_engine

In [2]:
# setup
engine = make_engine("DB")

pd.set_option("display.max_colwidth", 512)
pd.set_option("display.max_rows", 512)

colors = pd.DataFrame(
    [(label, LABELS[label]["color"]) for label in LABELS], columns=["labels", "color"]
)

In [3]:
df = load.samples_per_coder(engine, batch=1)

In [4]:
df_labels = df.explode("labels")
df_labels = df_labels[~df_labels.labels.str.startswith("no_")]
df_labels = pd.merge(df_labels, colors, on="labels", how="left")

# Agreement on Codes

- lists all examples in which at least 2 coders agree on at least one code
- shows only agreed-upon codes

In [6]:
d = df_labels.groupby(["sample_id", "labels"]).filter(lambda x: len(x) > 1)

visualize_dataframe(d.head(100), id_col="sample_id", color_col="color")

1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",coudry,antielite
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",grabsch,antielite
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",richter,antielite
,,,
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien aufgrund der Niedrigzinspolitik der Europäischen Zentralbank, fünftens der Verkauf von Wohnbeständen durch die Kommunen an renditeorientierte internationale Pensionskassen und Fonds, natürlich auf Kosten der Mieter – das waren Ihre Leute –, sechstens die häufige Zweckentfremdung der Mittel für den sozialen Wohnungsbau durch die Bundesländer.",grabsch,left
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien aufgrund der Niedrigzinspolitik der Europäischen Zentralbank, fünftens der Verkauf von Wohnbeständen durch die Kommunen an renditeorientierte internationale Pensionskassen und Fonds, natürlich auf Kosten der Mieter – das waren Ihre Leute –, sechstens die häufige Zweckentfremdung der Mittel für den sozialen Wohnungsbau durch die Bundesländer.",grabsch,antielite
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien aufgrund der Niedrigzinspolitik der Europäischen Zentralbank, fünftens der Verkauf von Wohnbeständen durch die Kommunen an renditeorientierte internationale Pensionskassen und Fonds, natürlich auf Kosten der Mieter – das waren Ihre Leute –, sechstens die häufige Zweckentfremdung der Mittel für den sozialen Wohnungsbau durch die Bundesländer.",grabsch,right
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge s

# Full Disagreement on codes

- This list shows only examples where at least two coders assigned codes, but the codes were completely different (no overlap)

In [8]:
def filter_disagreement(group):
    if len(group) < 2:
        return False

    labels = group.groupby("username")["labels"].agg(set)

    if len(labels) < 2:
        return False

    for i, j in it.combinations(labels, r=2):
        if i.intersection(j):
            return False

    return True


d = df_labels.groupby(["sample_id"]).filter(filter_disagreement)

visualize_dataframe(d, id_col="sample_id", color_col="color")

78940,"Diese innerhalb von zwei Jahren zurückzuzahlen, wie es die Regierung vorsieht, ist für viele Leute völlig realitätsfern, die jetzt schon 50 Prozent des Einkommens für das Wohnen ausgeben.",coudry,antielite
78940,"Diese innerhalb von zwei Jahren zurückzuzahlen, wie es die Regierung vorsieht, ist für viele Leute völlig realitätsfern, die jetzt schon 50 Prozent des Einkommens für das Wohnen ausgeben.",richter,pplcentr
,,,
194797,"Wir wollen erreichen, dass Recht, Gesetz und Verfassung überall und unbedingt gelten und nicht nach Belieben zurechtgebogen und ignoriert werden, dass Freiheit und Eigenverantwortung der Bürger im Mittelpunkt der Politik stehen und nicht Bevormundung in immer weitere Lebensbereiche.",richter,pplcentr
194797,"Wir wollen erreichen, dass Recht, Gesetz und Verfassung überall und unbedingt gelten und nicht nach Belieben zurechtgebogen und ignoriert werden, dass Freiheit und Eigenverantwortung der Bürger im Mittelpunkt der Politik stehen und nicht Bevormundung in immer weitere Lebensbereiche.",richter,eliteless
194797,"Wir wollen erreichen, dass Recht, Gesetz und Verfassung überall und unbedingt gelten und nicht nach Belieben zurechtgebogen und ignoriert werden, dass Freiheit und Eigenverantwortung der Bürger im Mittelpunkt der Politik stehen und nicht Bevormundung in immer weitere Lebensbereiche.",schadt,pplmore
,,,
265757,"Kolleginnen und Kollegen, ich bin oft der Meinung, wenn die Wählerinnen und Wähler wüssten, wie links die Grünen in der Sozialpolitik sind, würden viele von ihnen weglaufen.",coudry,antielite
265757,"Kolleginnen und Kollegen, ich bin oft der Meinung, wenn die Wählerinnen und Wähler wüssten, wie links die Grünen in der Sozialpolitik sind, würden viele von ihnen weglaufen.",coudry,right
265757,"Kolleginnen und Kollegen, ich bin oft der Meinung, wenn die Wählerinnen und Wähler wüssten, wie links die Grünen in der Sozialpolitik sind, würden viele von ihnen weglaufen.",richter,pplcentr
,,,


# Partial Disagreement on Codes

- shows codes that were given by one coder but not by others

In [10]:
def filter_disagreement(group):
    if (len(group) or group.label_by.nunique()) < 2:
        return False

    labels = group.groupby("username")["labels"].agg(set)

    for i, j in it.combinations(labels, r=2):
        if i != j:
            return True

    return False


d = df_labels.groupby(["sample_id"]).filter(filter_disagreement)

visualize_dataframe(d, id_col="sample_id", color_col="color")

1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",coudry,antielite
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",grabsch,antielite
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",richter,left
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",richter,antielite
1114410,"Wir kämpfen dagegen, dass Entscheidungen zunehmend nicht mehr hier im Parlament getroffen werden, sondern in Technokratenräte, in irgendwelche Kommissionen wie bei der Kohle verlagert werden, wo zwei oder drei Abgeordnete der Koalitionsfraktionen wie die Mitarbeiter hinten drinsitzen, nichts sagen und diesem Parlament auch nicht berichten dürfen.",schadt,eliteless
,,,
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien aufgrund der Niedrigzinspolitik der Europäischen Zentralbank, fünftens der Verkauf von Wohnbeständen durch die Kommunen an renditeorientierte internationale Pensionskassen und Fonds, natürlich auf Kosten der Mieter – das waren Ihre Leute –, sechstens die häufige Zweckentfremdung der Mittel für den sozialen Wohnungsbau durch die Bundesländer.",grabsch,left
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien aufgrund der Niedrigzinspolitik der Europäischen Zentralbank, fünftens der Verkauf von Wohnbeständen durch die Kommunen an renditeorientierte internationale Pensionskassen und Fonds, natürlich auf Kosten der Mieter – das waren Ihre Leute –, sechstens die häufige Zweckentfremdung der Mittel für den sozialen Wohnungsbau durch die Bundesländer.",grabsch,antielite
1039598,"Diese lauten: erstens die schon seit mehr als 15 Jahren andauernde stetige Flucht aus dem ländlichen Raum in die Städte, zweitens die gewollte Freizügigkeit seit 2011 für Arbeitnehmer und Selbstständige aus den osteuropäischen Staaten, die im Jahr 2004 der EU beigetreten sind, drittens die enorme Einwanderung von mehr als 1,8 Millionen meist illegaler sogenannter Flüchtlinge seit 2013 in unser Land, viertens die spekulativen Investitionen in Immobilien